In [ ]:
import queue
import json
import paho.mqtt.client as paho

mqtt_queue=queue.Queue()
def on_message(client, userdata, message):
  global mqtt_queue
  mqtt_queue.put(message)

client = paho.Client()
client.connect("lidar")
client.on_message = on_message
client.subscribe("homie/lidar_readout_0/data")
client.loop_start()

In [ ]:
#alles löschen
while not mqtt_queue.empty():
    message = mqtt_queue.get()


In [ ]:
import matplotlib.pyplot as plt
import numpy as np

In [ ]:
message=mqtt_queue.get()
m = message.payload.decode("utf-8")
data=json.loads(m)

In [ ]:
data = d[0]
# Separate angle and distance values
angles = np.array([entry[0] for entry in data])
distances = np.array([entry[1] for entry in data])

# Convert angles to radians and multiply by -1 for counterclockwise orientation
angles_rad = -(angles-100) * (2 * np.pi / 360)

# Calculate the average distance
avg_distance = np.mean(distances)

# Plot the data points and color the ones closer than the average distance differently
plt.figure()
plt.polar(angles_rad[distances < avg_distance], distances[distances < avg_distance], 'ro', markersize=3)  # Plot closer points in red
plt.polar(angles_rad[distances >= avg_distance], distances[distances >= avg_distance], 'bo', markersize=3)  # Plot other points in blue


plt.title('Polar Plot')
plt.show()

## Einsortierung in Array bins

Ab hier den Kernel neu starten

In [ ]:
from collections import deque
d = deque(maxlen=8) #8Lidar has Hz => 1Sec

reference_data = None

In [ ]:
import queue
import json
import paho.mqtt.client as paho

def on_message(client, userdata, message):
    global d
    m = message.payload.decode("utf-8")
    data=json.loads(m)
    d.append(data)

client = paho.Client()
client.connect("lidar")
client.on_message = on_message
client.subscribe("homie/lidar_readout_0/data")
client.loop_start()

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

In [ ]:
#prepare reference
reference_data = d[0]

# Separate angle and distance values
angles_ref = np.array([entry[0] for entry in reference_data])
distances_ref = np.array([entry[1] for entry in reference_data])

# Convert angles to radians and multiply by -1 for counterclockwise orientation
angles_rad_ref = -(angles_ref-210) * (2 * np.pi / 360)

In [ ]:
# Darstellung
while True:
    data = d[0]
    # Separate angle and distance values
    angles = np.array([entry[0] for entry in data])
    distances = np.array([entry[1] for entry in data])

    # Convert angles to radians and multiply by -1 for counterclockwise orientation
    angles_rad = -(angles-210) * (2 * np.pi / 360)

    # Plot the data points and color the ones closer than the average distance differently
    plt.figure()
    plt.polar(angles_rad, distances, 'ro', markersize=2)  # Plot closer points in red
    plt.polar(angles_rad_ref, distances_ref, 'bo', markersize=3)  # Plot closer points in blue


    plt.title('Polar Plot')
    plt.show()

# Unterschiede berechnen

In [ ]:
#to be flexible in using 180 or 360 etc. bins
reduction_factor = 2 # 2 for 180 degree bins, 1 for 360, etc.

In [24]:
# prepare variables: one bin for each of the full circle (360 degree)
d_combined = [] 
for v in range(360//reduction_factor):
    d_combined.append([])

#fill d_combined with data from d
for v1 in d:
    for v2 in v1:
      d_combined[int(v2[0]//reduction_factor)].append(v2[1])

# 5 measurements per degree bin necessary
for k, v1 in enumerate(d_combined):
    if len(v1) <5:
      d_combined[k] = None
      print(".", end="")
    else:
      d_combined[k] = sum(v1)/len(v1)
      print("X", end="")

XXXXXXXXXX................................................................XXXXX.........................................XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX

In [29]:
# set reference
reference_data = d_combined

In [30]:
import time
while True:
    # prepare variables: one bin for each of the full circle (360 degree)
    d_combined = [] 
    for v in range(360//reduction_factor):
        d_combined.append([])

    #fill d_combined with data from d
    for v1 in d:
        for v2 in v1:
          d_combined[int(v2[0]//reduction_factor)].append(v2[1])

    # 5 measurements per degree bin necessary
    for k, v1 in enumerate(d_combined):
        if len(v1) <5:
          d_combined[k] = None
        else:
          d_combined[k] = sum(v1)/len(v1)


    if reference_data:
      for k in range(len(reference_data)):
        if d_combined[k] and reference_data[k]:
          if d_combined[k] < reference_data[k]-100: # in mm
            print("_", end="")
          else:
            print(" ", end="")
        else:
          print(" ", end="")
    print("")
    time.sleep(1)

                                                                                                                                      _                                             
                                                                                                                                                                                    
                                                                                                                                                                                    
                                                                                                                                                                                    
                                                                                                                                                                                    
                                                                                               

KeyboardInterrupt: 